In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
from crewai import Agent, Crew, Task

In [6]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [8]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [9]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [10]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [11]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [12]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [13]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [14]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [15]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

In [16]:
event_details = {
    'event_topic': "Wedding event",
    'event_description': "A wedding event for 250 people with a budgt of 500 rs per plate ",
    'event_city': "Navi Mumbai",
    'tentative_date': "2025-03-05",
    'expected_participants': 250,
    'budget': 400000,
    'venue_type': "Hall or lawn"
}

In [17]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in Navi Mumbai that meets criteria for Wedding event.


> Entering new CrewAgentExecutor chain...
I need to start searching for a venue in Navi Mumbai that meets the criteria for a Wedding event.

Action: Search the internet
  Input: {"search_query": "Wedding venues in Navi Mumbai"}


Search results: Title: 20 Best Wedding Venues in Navi Mumbai - WedMeGood
Link: https://www.wedmegood.com/vendors/navi-mumbai/wedding-venues/
Snippet: List of best wedding venues in Navi Mumbai. Get best deals on Banquets, lawns & hotels. Check prices, contact number, reviews & more.
---
Title: The 10 Best Wedding Venues in Navi Mumbai - Weddingwire.in
Link: https://www.weddingwire.in/wedding-venues/mumbai/navi-mumbai
Snippet: Wedding Venues in Navi Mumbai - Compare and Find the Best Wedding Venues as per your wedding plan, Check menus, availability and services with ...
---
Title: Wedding Venues in Navi Mumbai | Marriott

This is the agent final answer: After searching the internet and reading the website content, I have found several wedding venues in Navi Mumbai that could accommodate a wedding event. Here are a few options:

1. Marriott Executive Apartments Navi Mumbai - 4 Star & Above Wedding Hotels, Banquet Halls
   - Vegetarian: ₹1,999 per plate, Non-vegetarian: ₹2,800 per plate
   - Capacity: 80-1000 guests
   - 128 Rooms available

2. Supreme Banquets - Banquet Halls, Small Function / Party Halls
   - Vegetarian: ₹1,000 per plate, Non-vegetarian: ₹1,200 per plate
   - Capacity: 100-800 guests
   - 2 Rooms available

3. The Cliff Resort and Spa - 4 Star & Above Wedding Hotels, Banquet Halls
   - Vegetarian: ₹1,500 per plate, Non-vegetarian: ₹1,500 per plate
   - Capacity: 112-250 guests
   - 41 Rooms available

4. The Forest Club Resort - 4 Star & Above Wedding Hotels, Banquet Halls
   - Vegetarian: ₹1,000 per plate, Non-vegetarian: ₹1,300 per plate
   - Capacity: 100-300 guests
   - 25 Rooms ava

Final Answer: 
After re-evaluating the information gathered from the internet search and website content, I have identified several wedding venues in Navi Mumbai that could accommodate a wedding event:

1. Marriott Executive Apartments Navi Mumbai - 4 Star & Above Wedding Hotels, Banquet Halls
   - Vegetarian: ₹1,999 per plate, Non-vegetarian: ₹2,800 per plate
   - Capacity: 80-1000 guests
   - 128 Rooms available

2. Supreme Banquets - Banquet Halls, Small Function / Party Halls
   - Vegetarian: ₹1,000 per plate, Non-vegetarian: ₹1,200 per plate
   - Capacity: 100-800 guests
   - 2 Rooms available

3. The Cliff Resort and Spa - 4 Star & Above Wedding Hotels, Banquet Halls
   - Vegetarian: ₹1,500 per plate, Non-vegetarian: ₹1,500 per plate
   - Capacity: 112-250 guests
   - 41 Rooms available

4. The Forest Club Resort - 4 Star & Above Wedding Hotels, Banquet Halls
   - Vegetarian: ₹1,000 per plate, Non-vegetarian: ₹1,300 per plate
   - Capacity: 100-300 guests
   - 25 Rooms available


Exception in thread Thread-6 (_execute):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/li

I need to gather detailed information about wedding services and packages offered at Marriott Executive Apartments Navi Mumbai.

Action: Read website content
Action Input: {"website_url": "https://www.marriott.com/hotels/travel/bomme-marriott-executive-apartments-mumbai"}
 

Access Denied
Access Denied
You don't have permission to access "http://www.marriott.com/en-us/hotels/bomme-marriott-executive-apartments-mumbai/overview/" on this server.
Reference #18.6601d517.1739520862.1ef2ba0
https://errors.edgesuite.net/18.6601d517.1739520862.1ef2ba0

Final Answer: Access Denied
Access Denied
You don't have permission to access "http://www.marriott.com/en-us/hotels/bomme-marriott-executive-apartments-mumbai/overview/" on this server.
Reference #18.6601d517.1739520862.1ef2ba0
https://errors.edgesuite.net/18.6601d517.1739520862.1ef2ba0

> Finished chain.
 

Access Denied
Access Denied
You don't have permission to access "http://www.marriott.com/en-us/hotels/bomme-marriott-executive-apartments-m

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Navi Mumbai',
 'booking_status': 'Available',
 'capacity': 1000,
 'name': 'Marriott Executive Apartments Navi Mumbai'}
